You are looking at data from an e-commerce website. The site is very simple and has just 4 pages:
1. The ﬁrst page is the home page. When you come to the site for the ﬁrst time, you can only land on the home page as a ﬁrst page.
2. From the home page, the user can perform a search and land on the search page.
3. From the search page, if the user clicks on a product, she will get to the payment page, where she is asked to provide payment information in order to buy that product.
4. If she does decide to buy, she ends up on the conﬁrmation page

The company CEO isn't very happy with the volume of sales and, especially, of sales coming from new users. Therefore, she asked you to investigate whether there is something wrong in the conversion funnel or, in general, if you could suggest how conversion rate can be improved.

In [210]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

# Index

# Load the data

In [187]:
allusers = pd.read_csv("home_page_table.csv",index_col="user_id")
users_to_search = pd.read_csv("search_page_table.csv",index_col="user_id")
users_to_pay = pd.read_csv("payment_page_table.csv",index_col="user_id")
users_to_confirm = pd.read_csv("payment_confirmation_table.csv",index_col="user_id")

In [188]:
allusers.loc[users_to_search.index,"page"] = users_to_search.page
allusers.loc[users_to_pay.index,"page"] = users_to_pay.page
allusers.loc[users_to_confirm.index,"page"] = users_to_confirm.page

In [189]:
# give it a better, more clear name
allusers.rename(columns={'page':'final_page'},inplace=True)

In [190]:
# change string to ordered-categorical feature
pages = ["home_page","search_page","payment_page","payment_confirmation_page"]
allusers["final_page"] = allusers.final_page.astype("category",categories = pages,ordered=True)

In [191]:
user_infos = pd.read_csv("user_table.csv",index_col="user_id")
user_infos.loc[:,"date"] = pd.to_datetime(user_infos.date)

In [192]:
allusers = allusers.join(user_infos)
allusers.head()

,final_page,date,device,sex
user_id,,,,
313593,home_page,2015-02-26,Desktop,Female
468315,home_page,2015-02-21,Desktop,Male
264005,home_page,2015-03-25,Desktop,Female
290784,home_page,2015-03-14,Desktop,Male
639104,search_page,2015-01-03,Desktop,Female


In [193]:
allusers.to_csv("all_users.csv",index_label="user_id")

# Answer question 1
<span style='color:blue;font-size:1.2em'>A full picture of funnel conversion rate for both desktop and mobile</span>

In [194]:
def conversion_rates(df):
    stage_counts = df.final_page.value_counts()
    # #users converts from current page
    convert_from = stage_counts.copy()

    total = df.shape[0]
    for page in stage_counts.index:
        n_left = stage_counts.loc[page]# how many users just stop at current page
        n_convert = total - n_left
        convert_from[page] = n_convert
        total = n_convert

    cr = pd.concat([stage_counts,convert_from],axis=1,keys=["n_drop","n_convert"])
    cr["convert_rates"] = cr.n_convert.astype(np.float)/(cr.n_drop + cr.n_convert)
    cr['drop_rates'] = 1 - cr.convert_rates

    return cr

In [195]:
allusers.groupby('device').apply(conversion_rates)

n_drop  n_convert  convert_rates  \
device                                                                
Desktop home_page                   30100      30100       0.500000   
        search_page                 27090       3010       0.100000   
        payment_page                 2860        150       0.049834   
        payment_confirmation_page     150          0       0.000000   
Mobile  home_page                   15100      15100       0.500000   
        search_page                 12080       3020       0.200000   
        payment_page                 2718        302       0.100000   
        payment_confirmation_page     302          0       0.000000   

                                   drop_rates  
device                                         
Desktop home_page                    0.500000  
        search_page                  0.900000  
        payment_page                 0.950166  
        payment_confirmation_page    1.000000  
Mobile  home_page                    0.500000  
        search_page                  0.800000  
        payment_page                 0.900000  
        payment_confirmation_page    1.000000

In [196]:
allusers.groupby('device')['final_page'].apply(lambda s: s.value_counts(normalize=True)).unstack()

,home_page,search_page,payment_page,payment_confirmation_page
device,,,,
Desktop,0.5,0.45,0.047508,0.002492
Mobile,0.5,0.40,0.090000,0.010000


# Answer question 2
<span style='color:blue;font-size:1.2em'>Some insights on what the product team should focus on in order to improve conversion rate as well as anything you might discover that could help improve conversion rate.</span>

In [197]:
allusers.head()

,final_page,date,device,sex
user_id,,,,
313593,home_page,2015-02-26,Desktop,Female
468315,home_page,2015-02-21,Desktop,Male
264005,home_page,2015-03-25,Desktop,Female
290784,home_page,2015-03-14,Desktop,Male
639104,search_page,2015-01-03,Desktop,Female


In [198]:
X = allusers.copy()

In [199]:
X.device.value_counts()

Desktop    60200
Mobile     30200
Name: device, dtype: int64

In [200]:
X['from_mobile'] = (X.device == 'Mobile').astype(int)
del X['device']

In [201]:
X['is_male'] = (X.sex == 'Male').astype(int)
del X['sex']

In [202]:
X['converted'] = (X.final_page == 'payment_confirmation_page').astype(int)
del X['final_page']

In [203]:
X.converted.mean()# a highly imbalanced classification problem

0.005

## Impact of date

In [204]:
X.date.describe()

count                   90400
unique                    120
top       2015-02-08 00:00:00
freq                      877
first     2015-01-01 00:00:00
last      2015-04-30 00:00:00
Name: date, dtype: object

In [205]:
X['weekday'] = X.date.dt.weekday_name
del X['date']

In [206]:
X.head()

,from_mobile,is_male,converted,weekday
user_id,,,,
313593,0,0,0,Thursday
468315,0,1,0,Saturday
264005,0,0,0,Wednesday
290784,0,1,0,Saturday
639104,0,0,0,Saturday


In [207]:
X.groupby('weekday')['converted'].agg(['count','mean']).sort_values(by='mean',ascending=False)

,count,mean
weekday,,
Monday,12955,0.005944
Saturday,12801,0.005468
Tuesday,12697,0.005434
Thursday,13444,0.004909
Sunday,12922,0.004798
Friday,12750,0.004235
Wednesday,12831,0.004209


## Impact of sex

In [208]:
X.groupby('is_male')['converted'].agg(['count','mean']).sort_values(by='mean',ascending=False)

,count,mean
is_male,,
0,45075,0.005347
1,45325,0.004655


## Statistical Test

In [209]:
X = pd.get_dummies(X,prefix='',prefix_sep='')
X.head()

,from_mobile,is_male,converted,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
user_id,,,,,,,,,,
313593,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
468315,0,1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
264005,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
290784,0,1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
639104,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [211]:
y = X.converted
X = X.loc[:,X.columns != 'converted']

In [213]:
scores, pvalues = chi2(X,y)

In [216]:
pd.DataFrame({'chi2_score':scores,'chi2_pvalue':pvalues},index=X.columns).sort_values(by='chi2_score',ascending=False)

,chi2_pvalue,chi2_score
from_mobile,7.153913e-35,151.758794
Monday,1.278169e-01,2.318821
Wednesday,2.037208e-01,1.615497
Friday,2.208761e-01,1.498670
is_male,2.980938e-01,1.082703
Saturday,4.525166e-01,0.564341
Tuesday,4.877434e-01,0.481501
Sunday,7.447867e-01,0.105964
Thursday,8.814147e-01,0.022253


In [182]:
del X['Tuesday']# remove one redundant feature

,from_mobile,is_male,converted,Friday,Monday,Saturday,Sunday,Thursday,Wednesday
user_id,,,,,,,,,
313593,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0
468315,0,1,0,0.0,0.0,1.0,0.0,0.0,0.0
264005,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
290784,0,1,0,0.0,0.0,1.0,0.0,0.0,0.0
639104,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0


In [184]:
dt = DecisionTreeClassifier(max_depth=3,min_samples_leaf=20,min_samples_split=20)
dt.fit(X,y)
export_graphviz(dt,feature_names=X.columns,class_names=['NotConvert','Converted'],
                proportion=True,leaves_parallel=True,filled=True)